In [2]:
import wikipediaapi
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

wiki_wiki = wikipediaapi.Wikipedia(extract_format=wikipediaapi.ExtractFormat.WIKI, user_agent='Your_User_Agent_String_Here')

def fetch_wikipedia_data(article_title):
    page = wiki_wiki.page(article_title)
    if page.exists():
        return page.text
    else:
        print(f"Page '{article_title}' does not exist.")

def construct_wikipedia_graph(seed_article, depth=1, max_nodes=50):
    visited = set()
    graph = nx.Graph()
    queue = [(seed_article, 0)]

    while queue:
        current_article, current_depth = queue.pop(0)
        visited.add(current_article)

        if current_depth < depth:
            try:
                page = wiki_wiki.page(current_article)
                links = page.links
                for title in sorted(links.keys()):
                    if len(visited) < max_nodes and title not in visited:
                        graph.add_edge(current_article, title)
                        queue.append((title, current_depth + 1))
            except wikipediaapi.WikipediaException as e:
                print(f"Error: {e}")

    return graph


seed_article = 'Graph_theory' 
wiki_data = fetch_wikipedia_data(seed_article)
graph = construct_wikipedia_graph(seed_article, depth=2, max_nodes=50)



/Users/blacksnow/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
import networkx as nx
from py2neo import Graph, Node, Relationship

# graph = nx.Graph()
# graph.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (4, 5)])

def import_to_neo4j(graph):
    uri = "bolt://localhost:7687"
    graph_db = Graph(uri)

    for node_id in graph.nodes():
        node = Node("NodeLabel", id=node_id)  
        graph_db.create(node)
    for source, target in graph.edges():
        source_node = graph_db.nodes.match("NodeLabel", id=source).first()
        target_node = graph_db.nodes.match("NodeLabel", id=target).first()
        graph_db.create(relationship := Relationship(source_node, "RELATED_TO", target_node))  

    print("Graph imported into Neo4j.")

import_to_neo4j(graph)

Graph imported into Neo4j.


In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv


dataset = Planetoid(root='.', name='Cora')

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * num_heads, out_channels, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(dataset.num_features, 32, dataset.num_classes, num_heads=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.NLLLoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(dataset[0].to(device))
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(dataset[0].to(device)).max(dim=1)
correct = int(pred[dataset.test_mask].eq(dataset.y[dataset.test_mask]).sum().item())
acc = correct / int(dataset.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')


Processing...
Done!


Test Accuracy: 0.8020


In [26]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [27]:
summary(model=model,input_size=(2708,1433))

AttributeError: 'Tensor' object has no attribute 'x'

In [16]:
print(model)

GAT(
  (conv1): GATConv(1433, 32, heads=4)
  (conv2): GATConv(128, 7, heads=1)
)


In [31]:
from torchviz import make_dot

make_dot(model, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

AttributeError: 'GAT' object has no attribute 'size'

In [29]:
!pip3 install torchviz

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.2 MB/s eta 0:00:00
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=bca6dc8f0d2073f1f4f2418dfeb0181524d2ccc0521549a5c40fa3f721fa98e9
  Stored in directory: /Users/blacksnow/Library/Caches/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
